SBC - CBR

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import pandas as pd

with open('/content/drive/MyDrive/3er/SBC/domini.json') as f:
  __domini = json.load(f)
noms = {
    'artistes': pd.Series(list(__domini['artistes'])),
    'periodes': pd.Series(list(__domini['periodes'])),
    'obres': pd.read_csv('/content/drive/MyDrive/3er/SBC/obres.csv').Titol
}

class Cas:
    def __init__(self, nombre, edat, t_dia, dies, artistes, periodes):
        if len(artistes) > 0 and isinstance(artistes[0], str):
            artistes = noms['artistes'].isin(artistes).to_numpy()
        if len(periodes) > 0 and isinstance(periodes[0], str):
            periodes = noms['periodes'].isin(periodes).to_numpy()

        self.nombre = nombre
        self.edat = edat
        self.temps = t_dia * dies
        self.dies = dies
        self.artistes = artistes
        self.periodes = periodes
        self.valoracio = None
        self.obres = None

    @property
    def classificadors(self):
        return self.nombre, self.edat, self.temps

    @property
    def noms_artistes(self):
        return list(noms['artistes'][self.artistes])

    @property
    def noms_periodes(self):
        return list(noms['periodes'][self.periodes])

    @property
    def noms_obres(self):
        return list(noms['obres'][self.obres])

    @staticmethod
    def guardar(casos):
        casos = [
            [cas.nombre, cas.edat, cas.temps, cas.dies,
             *cas.artistes, *cas.periodes, *cas.obres, cas.valoracio]
        for cas in casos]
        casos = np.array(casos)
        np.save('dades/casos.npy', casos)

    @staticmethod
    def carregar():
        casos = np.load('dades/casos.npy')
        stack = []
        for cas in casos:
            stack.append(Cas(
                cas[0].astype(int),
                cas[1].astype(int),
                cas[2].astype(int),
                cas[3].astype(int),
                cas[4:12].astype(bool),
                cas[12:92].astype(bool)
            ))
            stack[-1].obres = cas[92:-1].astype(bool)
            stack[-1].valoracio = cas[-1]
        return stack

In [5]:
import numpy as np

layer_th = [
    [2, 4, 8],        # nombre
    [16, 65], # edat
    [180, 360, 600, 1200],  # temps
]

class Arbre:
    def __init__(self, i=0, maxsize=10000):
        """
        Inicialitza l'arbre amb una profunditat 'i' i un tamany de fulla 'maxsize'.

        :param i: profunditat de l'arbre (0 per defecte)
        :param maxsize: tamany de fulla (10000 per defecte)
        """
        if i < len(layer_th):
            self.children = [Arbre(i+1) for _ in range(len(layer_th) + 1)]
        else:
            self.casos = set()
        self.i = i
        self.maxsize = maxsize

    def __search_leaf(self, cas):
        if self.i < len(layer_th):
            for th, child in zip(layer_th[self.i], self.children):
                if cas.classificadors[self.i] < th:
                    return child.__search_leaf(cas)
            return self.children[-1].__search_leaf(cas)
        else:
            return self

    def fetch(self, cas):
        leaf = self.__search_leaf(cas)
        return leaf.casos

    def feed(self, cas):
        leaf = self.__search_leaf(cas)
        leaf.casos.add(cas)

    def recorre_fulles(self):
        casos = set()
        if self.i < len(layer_th):
            for child in self.children:
                casos |= child.recorre_fulles()
        else:
            casos |= self.casos
        return casos

    def __mantenir(self, v1, v2):
        if self.i < len(layer_th):
            casos_inutils = set()
            for child in self.children:
                casos_inutils |= child.__mantenir(v1, v2)
            return casos_inutils
        else:
            casos_inutils = {cas for cas in self.casos if v1 < cas.valoracio < v2}
            self.casos -= casos_inutils
            return casos_inutils

    def mantenir(self):
        casos = self.recorre_fulles()
        while len(casos) > self.maxsize:
            v1, v2 = np.percentile([cas.valoracio for cas in casos], [49,51])
            casos -= self.__mantenir(v1, v2)

In [6]:
import numpy as np
import json
import pandas as pd

rng = np.random.default_rng()

with open('/content/drive/MyDrive/3er/SBC/domini.json') as f:
    domini = json.load(f)

obres = pd.read_csv('/content/drive/MyDrive/3er/SBC/obres.csv')

def valorar(cas):
    # preferencies
    no_pref = ~(obres.Artista.isin(cas.noms_artistes) | obres.Periode.isin(cas.noms_periodes))
    obres_no_pref = obres[cas.obres & no_pref]

    preferencies = set(domini['periodes'][p] for p in cas.noms_periodes)
    periode_artistes = set(domini['artistes'][a] for a in cas.noms_artistes)
    preferencies.update(domini['periodes'][p] for p in periode_artistes)
    preferencies = np.array(list(preferencies))
    recomanacio = np.array([domini['periodes'][p] for p in obres_no_pref.Periode])
    if preferencies.size > 0 and recomanacio.size > 0:
        dist = np.abs(recomanacio[:, None] - preferencies).min(-1)
        dist = dist.mean()
    else:
        dist = 0.

    cas.valoracio = -dist / 7 + 1


def recomanar_random(casos):
    t = np.array([c.temps for c in casos])
    t -= obres.Sala.max() * 2
    n_obres_aprox = t / obres.Temps.mean()
    p = np.clip(n_obres_aprox / obres.shape[0], None, 1)
    p = np.tile(p, (obres.shape[0], 1)).T
    recomanacio = rng.binomial(1, p).astype(bool)

    for c, r in zip(casos, recomanacio):
        c.obres = r


def generar_casos(n):
    # MIDA
    mida = rng.choice(('individu', 'parella', 'grup', 'gran'), n)
    nombre = np.empty(n, int)
    nombre[mida == 'individu'] = 1
    nombre[mida == 'parella'] = 2
    nombre[mida == 'grup'] = rng.integers(3, 8, (mida == 'grup').sum())
    nombre[mida == 'gran'] = rng.integers(8, 16, (mida == 'gran').sum())

    # EDAT
    edat = rng.normal(50, 15, n).clip(5, 95).round().astype(int)
    generacio = pd.cut(edat, [4, 10, 20, 35, 65, 96],
                       labels=['infant', 'adolescent', 'jove', 'adult', 'juvilat'])

    # T_DIA
    t_dia = (np.log(edat) - np.log(5)) * (540 - 120) / (np.log(95) - np.log(5)) + 120
    t_dia += rng.normal(0, 60, n)
    t_dia = t_dia.clip(60, 600).round().astype(int)

    # DIES
    p = generacio.map({
        'infant': 0.2,
        'adolescent': 0.25,
        'jove': 0.3,
        'adult': 0.35,
        'juvilat': 0.4
    })
    dies = 1 + rng.binomial(2, p)

    # ARTISTES
    artistes = rng.binomial(1, 1/len(noms['artistes']), (n, len(noms['artistes'])))

    artistes_pop = (
        "Rembrandt (Rembrandt van Rijn)",
        "Peter Paul Rubens",
        "Jean Honore Fragonard",
        "Goya (Francisco de Goya y Lucientes)",
        "Eugene Delacroix",
        "Jean-Francois Millet",
        "Claude Monet",
        "Vincent van Gogh",
        "Edgar Degas"
    )
    artistes[:, np.isin(noms['artistes'], artistes_pop)] = \
        rng.binomial(1, 1/len(artistes_pop), (n, len(artistes_pop)))

    artistes = artistes.astype(bool)

    # PERIODES
    periodes = rng.binomial(1, 1/len(noms['periodes']), (n, len(noms['periodes'])))

    periodes_pop = (
        "Renaissance",
        "Impressionism"
    )
    periodes[:, np.isin(noms['periodes'], periodes_pop)] = \
        rng.binomial(1, 1/len(periodes_pop), (n, len(periodes_pop)))

    periodes = periodes.astype(bool)

    return [Cas(*feats) for feats in zip(
        nombre,
        edat,
        t_dia,
        dies,
        artistes,
        periodes
    )]

In [7]:
import random

with open("/content/drive/MyDrive/3er/SBC/domini.json") as f:
    domini = json.load(f)
obres = pd.read_csv('/content/drive/MyDrive/3er/SBC/obres.csv')

class CBR:
    def __init__(self, arbre, artist_weight=1., period_weight=1., age_weight=1., time_weight=1.):
        """
        Inicialitza el sistema amb l'arrel i els pesos per artistes, periodes, edat i hores.
        """
        self.arbre = arbre
        self.artist_weight = artist_weight
        self.period_weight = period_weight
        self.age_weight = age_weight
        self.time_weight = time_weight

    def calculate_distance(self, case, leaf_case):
        """
        Calcula la distància entre dos casos, considerant artistes, periodes, edat i hores.
        """
        # Distància conjunta per artistes i periodes (Jaccard combinada)
        artists1, artists2 = set(leaf_case.noms_artistes), set(case.noms_artistes)
        periods1, periods2 = set(leaf_case.noms_periodes), set(case.noms_periodes)

        # Jaccard per artistes
        artist_similarity = len(artists1 & artists2) / len(artists1 | artists2) if artists1 | artists2 else 0
        artist_distance = 1 - artist_similarity

        # Jaccard per periodes
        period_similarity = len(periods1 & periods2) / len(periods1 | periods2) if periods1 | periods2 else 0
        period_distance = 1 - period_similarity

        # Distància per edat (normalitzada)
        age_distance = abs(leaf_case.edat - case.edat) / max(leaf_case.edat, case.edat)

        # Distància per hores (normalitzada)
        time_distance = abs(leaf_case.temps - case.temps) / max(leaf_case.temps, case.temps)

        # Combinar artistes, periodes, edat i hores segons els pesos
        combined_distance = (
            artist_distance * self.artist_weight +
            period_distance * self.period_weight +
            age_distance * self.age_weight +
            time_distance * self.time_weight
        ) / (self.artist_weight + self.period_weight + self.age_weight + self.time_weight)

        return combined_distance

    def retrieve(self, case):
        """
        Busca els 5 casos més propers en el sistema de casos, considerant artistes, periodes, edat i
        hores.
        """
        leaf_cases = self.arbre.fetch(case)  # Recuperem tots els casos de la fulla
        if len(leaf_cases) == 0:
            return []
        else:
            # Calcular distàncies per a tots els casos de les fulles
            distances = [
                (leaf_case, self.calculate_distance(case, leaf_case))
                for leaf_case in leaf_cases
            ]

            # Ordenar els casos per distància (de menor a major)
            distances.sort(key=lambda x: x[1])

            # Seleccionar els 5 millors casos
            top_cases = distances[:5]

            # Retornar els millors casos
            return top_cases

    def reuse(self, casospropers, case):
        """
        Adapta la informació del cas recuperat per crear una solució inicial pel nou cas.
        """
        # Prioritzar preferències
        recomanacio = obres.Artista.isin(case.noms_artistes)
        recomanacio |= obres.Periode.isin(case.noms_periodes)
        recomanacio = recomanacio.to_numpy()

        temps_acumulat = obres[recomanacio].Temps.sum()

        if temps_acumulat > case.temps:
            pass

        # Agafar obres dels casos propers
        punts_obres = np.zeros(obres.shape[0])
        for cas_prop, dist in casospropers:
            pes = cas_prop.valoracio * 2 - 1 # Passar a [-1, 1]
            pes = pes * dist
            puntuacions = cas_prop.obres * pes
            punts_obres += puntuacions

        # Softmax
        probs_obres = np.exp(punts_obres - punts_obres.max())
        probs_obres /= probs_obres.sum()

        probs_obres[recomanacio] = 0.
        probs_obres /= probs_obres.sum()

        while temps_acumulat < case.temps and recomanacio.sum() < obres.shape[0]:
            o = random.choices(range(obres.shape[0]), probs_obres)[0]
            recomanacio[o] = True
            probs_obres[o] = 0.
            probs_obres /= probs_obres.sum()
            temps_acumulat += obres.iloc[o].Temps

        case.obres = recomanacio

    def retain(self, cas):
        """
        Emmagatzema el nou cas i la seva solució al sistema de casos.
        """
        tots_casos = self.arbre.recorre_fulles()
        v25, v75 = np.percentile([cas.valoracio for cas in tots_casos], [25, 75])
        if cas.valoracio < v25 or cas.valoracio > v75:
            self.arbre.feed(cas)


    def __call__(self, case):
        """
        Implementa tot el cicle CRB per un cas donat.
        """
        # 1. Retrieve
        top_cases = self.retrieve(case)

        # 2. Reuse
        self.reuse(top_cases, case)

        # 3. Valorar
        valorar(case)

        # 4. Retain
        self.retain(case)

In [12]:
import matplotlib.pyplot as plt
import numpy as np

print('INICIALITZACIÓ')
casos_inicials = generar_casos(1000)
recomanar_random(casos_inicials)
for cas in casos_inicials: valorar(cas)

arbre = Arbre()
for cas in casos_inicials: arbre.feed(cas)

print('ENTRENAMENT')
cbr = CBR(arbre)

print('\n=== Entrenament ===')
casos_entrenament = generar_casos(1000)
valoracions_entrenament = []
for i, cas in enumerate(casos_entrenament, 1):
    cbr(cas)
    if i % 100 == 0:
        arbre.mantenir()
    valoracions_entrenament.append(cas.valoracio)
    print('\r', i, '/', len(casos_entrenament), end='')
print()

INICIALITZACIÓ


<ipython-input-6-d204a1024609>:63: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  p = generacio.map({


ENTRENAMENT

=== Entrenament ===
 6 / 1000

<ipython-input-6-d204a1024609>:63: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  p = generacio.map({


 1000 / 1000


In [14]:
def crear_cas():
    # Preguntes a l'usuari
    nombre = int(input("Quantes persones feu la visita? "))  # Assegura que sigui un enter
    edat = int(input("Quina és l'edat més representativa del grup? "))
    t_dia = float(input("Quant temps tens per fer la visita (per dia i en minuts)? "))
    dies = int(input("Quants dies durarà la teva visita? "))

    # Preguntes per artistes i periodes
    artistes_input = input("Quins artistes vols incloure? (Separats per comes) ")
    periodes_input = input("Quins períodes vols incloure? (Separats per comes) ")

    # Convertir les respostes a llistes
    artistes = artistes_input.split(",") if artistes_input else []
    periodes = periodes_input.split(",") if periodes_input else []

    # Comprovar si els artistes i periodes són correctes dins del domini
    artistes_valids = ["Vincent van Gogh", "Claude Monet", "Leonardo da Vinci", "Anthony van Dyck", "Rembrandt (Rembrandt van Rijn)", "Frans Hals"]
    periodes_valids = ["Renaissance", "Baroque", "Realism", "Impressionism"]

    artistes = [art.strip() for art in artistes if art.strip() in artistes_valids]
    periodes = [per.strip() for per in periodes if per.strip() in periodes_valids]

    # Crear la instància de la classe Cas
    cas = Cas(nombre, edat, t_dia, dies, artistes, periodes)

    # Debug per verificar les dades
    print("Dades del cas creat:", cas.__dict__)

    return cas

# Crear un cas
cas1 = crear_cas()

Quantes persones feu la visita? 3
Quina és l'edat més representativa del grup? 20
Quant temps tens per fer la visita (per dia i en minuts)? 300
Quants dies durarà la teva visita? 3
Quins artistes vols incloure? (Separats per comes) 
Quins períodes vols incloure? (Separats per comes) 
Dades del cas creat: {'nombre': 3, 'edat': 20, 'temps': 900.0, 'dies': 3, 'artistes': [], 'periodes': [], 'valoracio': None, 'obres': None}
Classificadors: (3, 20, 900.0)


In [19]:
# Executar el retrieve per obtenir els casos més semblants
top_casos = cbr.retrieve(cas1)

# Reutilitzar els casos i obtenir les obres seleccionades
obres_visitar = cbr.reuse(top_casos, cas1)

# Mostrar les obres seleccionades per visitar
print("\nObres seleccionades per visitar en el cas:")
for obra in cas1.noms_obres:
    print(f"- {obra}")

# Preguntar a l'usuari per introduir una valoració numèrica
valoracio = float(input("\nIntrodueix una valoració per aquesta proposta (ex: 1-10): "))

# Assignar la valoració al cas
cas1.valoracio = valoracio

# Fer el retain per afegir el cas amb la seva valoració i obres al sistema
cbr.retain(cas1)

print("\nEl cas s'ha guardat correctament al sistema amb la nova valoració i obres seleccionades.")


Obres seleccionades per visitar en el cas:
- The Life of the Virgin
- Man Weighing Gold
- Saint Rosalie Interceding for the Plague-stricken of Palermo
- Study Head of an Old Man with a White Beard
- Virgin and Child with Saint Catherine of Alexandria
- A Man Mounting a Horse
- Queen Henrietta Maria
- James Stuart (1612-1655), Duke of Richmond and Lennox
- Lucas van Uffel (died 1637)
- Still Life with Peaches
- Tilla Durieux (Ottilie Godeffroy, 1880-1971)
- A Lane through the Trees
- Reverie
- The Gypsies
- The Public Garden at Pontoise
- Two Young Peasant Women
- A Cowherd at Valhermeil, Auvers-sur-Oise
- Warwick Castle
- The Grand Canal, Venice, Looking South toward the Rialto Bridge
- Imaginary View with a Tomb by the Lagoon
- Apple Blossoms
- The Seine: Morning
- Portejoie on the Seine
- The Banks of the Oise
- The Green Wave
- Water Lilies
- Bouquet of Sunflowers
- Tobias and the Angel
- Selvaggia Sassetti (born 1470)
- Two Men
- Dancer
- Woman on a Sofa
- Woman with a Towel
- Dan